In [7]:
import experiment_settings as eset

# Перекидываем картинки в папку эксперимента

In [51]:
input_img_dir = 'D:\_github\experiments\\all_imgs'
output_img_dir = 'D:\_github\experiments\\filitzin\\all_cut_imgs\\'
eset.scale_images(input_dir=input_img_dir, output_dir=output_img_dir)

# Настройка csv

In [9]:
import pandas as pd

In [22]:
def category_filter(categories: list, actions: list, out_csv_path: str,
                               csv_path: str):
    
    """ 
    Метод подготовки csv-файла для эксперимента с метками-категориями
    categories: список строк с названиями категорий, по которым
                будет проводиться эксперимент
                out_csv_path: путь к новому csv файлу
    csv_path: путь к старому csv файлу
    is_action: True - лейблы в столбце action,False - лейблы в столбце category
    """
    
    df = pd.read_csv(csv_path, names = ['name', 'train', 'action', 
                                                'category'])
    
    """
    Предварительное удаление лишних столбцов и строк не из тестовой выборки
    """
    df_remove_test = df[df['train']==0]
    df.drop(df_remove_test.index, axis =0, inplace = True)
    
    df_action=df.copy()
    
    df.drop(['train','action'], axis =1, inplace =True)
    df = df.query('category in @categories')
    df=df.rename(columns = {'category':'label'})
    # Удаляем столбец категорий
    
    
    df_action.drop(['train','category'], axis =1, inplace =True)
    df_action = df_action.query('action in @actions')
    df_action = df_action.rename(columns = {'action':'label'})
    
    
    print('------------------------------------------------')
    print(df_action)
    print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
    
    frames = [df, df_action]
    df=pd.concat(frames)    
    
    """
    Замена категорий/действий на числовые лейблы, начиная с 0 и без пропусков
    """
    new_labels = {}
    label_counter = 0
    for category in categories:
        new_labels[category] = label_counter
        label_counter+=1
    for action in actions:
        new_labels[action] = label_counter
        label_counter+=1
    print(df)
    df = df.replace(new_labels)
    print('?????????????????????????????????????')
    print(df)
    
    df.to_csv(out_csv_path, header = True, index = False)

In [23]:
categories = ['walking', 'running']
actions = ['2', '103', '227']
out_csv_path = 'D:\_github\experiments\\filitzin\\filitzin.csv'
input_csv_path = 'D:\_github\experiments\mpii9.csv'
category_filter(categories=categories, actions=actions, out_csv_path=out_csv_path, csv_path=input_csv_path)

------------------------------------------------
                name label
47     031171108.jpg     2
624    048452299.jpg     2
818    039900270.jpg     2
1868   016481482.jpg   103
1870   072597991.jpg   103
...              ...   ...
18797  087410963.jpg   227
20254  061548842.jpg   103
21484  093677207.jpg     2
21962  063042646.jpg     2
24353  097843422.jpg   103

[197 rows x 2 columns]
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
                name    label
335    004600264.jpg  walking
336    004072880.jpg  walking
387    028203457.jpg  walking
388    017743954.jpg  walking
474    089114311.jpg  running
...              ...      ...
18797  087410963.jpg      227
20254  061548842.jpg      103
21484  093677207.jpg        2
21962  063042646.jpg        2
24353  097843422.jpg      103

[1121 rows x 2 columns]
?????????????????????????????????????
                name  label
335    004600264.jpg      0
336    004072880.jpg      0
387    028203457.jpg      0
388    017743954.jp

# Отбор картинок по csv

In [58]:
import os
from shutil import copy

def get_files_by_labels(csv_path:str, labels_and_counts:list, 
                          imgs_source_dir:str):
    
    df = pd.read_csv(csv_path, names = ['name', 'label'])
    raws = df.shape[0]
    print(raws)
    file_names = []
    
    for label, base_count in labels_and_counts: 
        count = base_count
        for i in range(1, raws):
            if count <= 0:
                break
            if os.path.exists(imgs_source_dir+df['name'][i]):
                #print('exist')
                if (int(df['label'][i]) == label):
                    count -= 1
                    file_names.append(df['name'][i])
        print(str(label)+": " + str(base_count - count))

    print("NAMEEEEEES")   
    print(len(file_names))
        
    return file_names

def copy_files(source_dir: str, destination_dir: str, files: list):
    
    fails = []
    for file in files:
        try:
            copy(source_dir+file, destination_dir)
        except FileNotFoundError:
            fails.append(file)
    return fails

In [59]:
csv_path = 'D:\_github\experiments\\filitzin\\filitzin.csv'
labels_and_counts = [(0, 90),(1, 90),(2, 90),(3, 90),(4, 90)]
imgs_source_dir = 'D:\_github\experiments\\filitzin\\all_cut_imgs\\'
imgs_target_dir = 'D:\_github\experiments\\filitzin\\1_max90\\'
print(imgs_target_dir)
files = get_files_by_labels(csv_path=csv_path, labels_and_counts=labels_and_counts, imgs_source_dir=imgs_source_dir)
#imgs_source_dir = 'D:\_github\experiments\\filitzin\\all_cut_imgs'
copy_files(source_dir=imgs_source_dir, destination_dir=imgs_target_dir, files=files)

D:\_github\experiments\filitzin\1_max90\
1122
0: 90
1: 90
2: 34
3: 73
4: 90
NAMEEEEEES
377


[]